In [1]:
%pip install spacy nltk 

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip list

Package                           Version
--------------------------------- ------------
aext_assistant                    0.4.0
aext_assistant_server             0.4.0
aext_core                         0.4.0
aext_core_server                  0.4.0
aext_shared                       0.4.0
aiobotocore                       2.7.0
aiofiles                          22.1.0
aiohappyeyeballs                  2.4.3
aiohttp                           3.10.10
aioitertools                      0.7.1
aiosignal                         1.2.0
aiosqlite                         0.18.0
alabaster                         0.7.12
alembic                           1.13.3
altair                            5.0.1
anaconda-anon-usage               0.4.3
anaconda-catalogs                 0.2.0
anaconda-client                   1.12.3
anaconda-cloud-auth               0.5.1
anaconda-navigator                2.6.0
anaconda-project                  0.11.1
annotated-types                   0.7.0
anyio                  

In [6]:
import re
import json
import spacy
import nltk
from nltk import CFG, ChartParser
from nltk.tokenize import word_tokenize

In [7]:
# Download NLTK data if not already done

nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/safamohammed/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/safamohammed/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [72]:
import pandas as pd 

In [86]:
VLMresponses = pd.read_csv("VLM_responses_Baseline.csv")
VLMresponses.head()

,UserPrompt,AssistantResponse,JSON,Accurate,Why it Failed?
0,I want to view traffic updates.,The user wants to view traffic updates. To com...,"{\n ""task"":""view traffic updates"",\n ""butt...",TRUE,NaN
1,I want to change the contact name to Jane Do.,To change the contact name to Jane Do using th...,"{\n ""task"":""change contact name to Jane Do"",...",TRUE,NaN
2,I want to add a red bag to my bag.,"Based on the image, the user would need to fol...",NaN,FALSE,NaN
3,I want to access my lessons offline.,"Based on the image, the user can complete the ...",NaN,FALSE,NaN
4,I want to give my answer verbally.,The image shows a mobile device displaying a u...,"{\n ""task"":""answer verbally and submit"",\n ...",TRUE,NaN


In [87]:
VLMresponses = VLMresponses[VLMresponses["Accurate"] != "FALSE"]

In [88]:
VLMresponses.head()

,UserPrompt,AssistantResponse,JSON,Accurate,Why it Failed?
0,I want to view traffic updates.,The user wants to view traffic updates. To com...,"{\n ""task"":""view traffic updates"",\n ""butt...",TRUE,NaN
1,I want to change the contact name to Jane Do.,To change the contact name to Jane Do using th...,"{\n ""task"":""change contact name to Jane Do"",...",TRUE,NaN
4,I want to give my answer verbally.,The image shows a mobile device displaying a u...,"{\n ""task"":""answer verbally and submit"",\n ...",TRUE,NaN
5,Can I maintain my current weight?,"Based on the image, to complete the task of 'C...","{\n ""task"":""maintain current weight"",\n ""b...",TRUE,NaN
6,Can you save my location?,"To save the user's location, the user should f...","{\n ""task"":""save location"",\n ""buttons"":[\...",TRUE,NaN


In [89]:
VLMresponses = VLMresponses.drop("Why it Failed?", axis='columns')
VLMresponses = VLMresponses.reset_index(drop=True)
VLMresponses.head()

,UserPrompt,AssistantResponse,JSON,Accurate
0,I want to view traffic updates.,The user wants to view traffic updates. To com...,"{\n ""task"":""view traffic updates"",\n ""butt...",TRUE
1,I want to change the contact name to Jane Do.,To change the contact name to Jane Do using th...,"{\n ""task"":""change contact name to Jane Do"",...",TRUE
2,I want to give my answer verbally.,The image shows a mobile device displaying a u...,"{\n ""task"":""answer verbally and submit"",\n ...",TRUE
3,Can I maintain my current weight?,"Based on the image, to complete the task of 'C...","{\n ""task"":""maintain current weight"",\n ""b...",TRUE
4,Can you save my location?,"To save the user's location, the user should f...","{\n ""task"":""save location"",\n ""buttons"":[\...",TRUE


In [90]:
# Sample Input
text = VLMresponses.iloc[22]["AssistantResponse"]
print(text)

jsonForm = VLMresponses.iloc[22]["JSON"]
print(jsonForm)


To complete the task of logging in with current credentials, the user should follow these steps based on the visible UI elements:

1. Tap on the "Enter your SplashTop Account" button to enter the email field.
2. Type in the email associated with the SplashTop account.
3. Tap on the "Password" field to enter the password.
4. Type in the password associated with the SplashTop account.
5. Tap on the "Log in" button to log in with the entered credentials.
{
   "task":"login with my current credentials",
   "buttons":[
      "Enter your SplashTop Account",
      "Log in"
   ],
   "field":[
      "password",
      "email"
   ]
}


In [104]:
def improved_semantic_parser(text):
    if not isinstance(text, str):
        return None

    # Try different ways to extract the task
    task_patterns = [
        r"wants to ([a-zA-Z\s]+@?)(?:.|\n|$)",
        r"To ([a-zA-Z\s]+@?)(?:.|\n|$)",
        r"task of ['\"]?([a-zA-Z\s]+@?)['\"]?(?:.|\n|$)",
        r"complete the task of ['\"]?([a-zA-Z\s]+@?)['\"]?(?:.|\n|$)",
        r"user (?:can|should) ([a-zA-Z\s]+@?)(?:.|\n|$)"
    ]

    task = None
    for pattern in task_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            task = match.group(1).strip().rstrip(".")
            break
        
    button_patterns = [
        r"Tap on the (.*?) button",
        r"Tap on the (.*?) icon",
        r"Tap on the (.*?) option",
        r"Press the (.*?) button",
    ]
    
    buttons = set()
    # Extract quoted UI elements as buttons
    for pattern in button_patterns:
        match = re.findall(pattern, text)
        
        if match:
            match = [m.strip().strip(r"'|\"") for m in match ] #if 1 <= len(m.strip().split()) <= 4
            for m in match:
                buttons.add(m.strip())
    
    fields = set()
    input_patterns = [
        r"\b(\w+)\s+field",
        r"(\S+)\s+field",
    ]
    
    for pattern in input_patterns:
        match = re.findall(pattern, text)
        if match:
            for m in match:
                fields.add(m.strip().strip(r"'|\""))
    

    return {
        "task": task,
        "buttons": list(buttons),
        "field": list(fields)
    }

In [105]:
print("\n=== Semantic Parser Output ===")
print(json.dumps(improved_semantic_parser(text), indent=3))


=== Semantic Parser Output ===
{
   "task": "complete the task of logging in with current credentials",
   "buttons": [
      "Enter your SplashTop Account",
      "Log in"
   ],
   "field": [
      "email",
      "Password"
   ]
}
